In [149]:
import pandas as pd, numpy as np, json, os

In [150]:
#os.chdir('E:/Onedrive - Lancaster University/datarepo/influence/ro')
os.chdir('C:/users/csala/Onedrive - Lancaster University/datarepo/influence/ro')

In [151]:
nmembers=json.loads(open('data/nmembers.json','r').read())

In [152]:
img=json.loads(open('data/img.json','r').read())

In [153]:
deaths=json.loads(open('data/deaths.json','r').read())

In [154]:
#os.chdir('E:/Skydrive/Github/influence/ro')
os.chdir('C:/users/csala/Onedrive/Github/influence/ro')

In [155]:
legs={'2016':['2016-12-21','2019-07-01'],
'2012':['2012-12-20','2016-12-21'],
'2008':['2008-12-19','2012-12-20'],
'2004':['2004-12-19','2008-12-13'],
'2000':['2000-12-15','2004-11-30'],
'1996':['1996-11-27','2000-11-30'],
'1992':['1992-10-28','1996-11-22'],
'1990':['1990-06-19','1992-10-16']}

In [158]:
import altair as alt

In [159]:
from colorthief import ColorThief
import requests

In [160]:
party_imgs={}
for p in img['party']:
    i=img['party'][p]['img']
    try: 
        color_thief = ColorThief('img/'+p.lower()+'.png')
        rgb_color=color_thief.get_color(quality=1)        
        z='#%02x%02x%02x' % rgb_color
    except:
        try:
            color_thief = ColorThief('img/'+p.lower()+'.jpg')
            rgb_color=color_thief.get_color(quality=1)        
            z='#%02x%02x%02x' % rgb_color
        except:
            z='grey'  
    party_imgs[p]=z

In [161]:
party_imgs['UDMR']='green'
party_imgs['RMDSZ']='green'
party_imgs['PNL']='#f4d42c'
party_imgs['ALDE']='#046cab'

In [162]:
colors=pd.DataFrame(party_imgs,index=['Color']).T

In [163]:
plinks={}
for i in colors.index:
    print('"'+i+'":"'+colors.loc[i]['Color']+'",')
    plinks[i]='https://influence.csaladen.es/ro/img/'+i.lower()+'.png'

"PUR-SL":"grey",
"PC":"#96c2f2",
"PSD":"#dc2c24",
"PUNR":"grey",
"PDL":"grey",
"PNL":"#f4d42c",
"PP-DD":"#d7c9e0",
"FSN":"grey",
"PDSR":"#ebedf1",
"PNTCD":"#04843c",
"PRM":"#0d5ca4",
"PD":"grey",
"UDMR":"green",
"PAC":"grey",
"PER":"grey",
"PMP":"#0484cc",
"USR":"#04abe4",
"UNPR":"#a8190f",
"FDSN":"grey",
"FER":"grey",
"PSM":"grey",
"PNL-CD":"grey",
"ALDE":"#046cab",
"FC":"#1d7aad",
"PL'93":"grey",
"PDAR":"grey",
"Fără adeziune la formaţiunea politică pentru care a candidat la alegeri":"grey",
"MER":"grey",
"AUR":"grey",
"PSDR":"#2e2f26",
"GDC":"grey",
"PAR":"grey",
"PLS":"grey",
"ULB":"grey",
"PRNR":"grey",
"PNL-AT":"grey",
"PDM":"grey",
"PTLDR":"grey",
"Minorități":"#eeeeee",
"Independent":"#eeeeee",
"RMDSZ":"green",


Members

In [164]:
szen=['🔼 Szenátus','🔽 Képviselőház']
szen2=['Senat','Camera Deputaților']

In [165]:
istart=legs['1990'][0]
iend=legs['2016'][1]

In [166]:
m=nmembers['Popescu-Tariceanu Calin Constantin Anton | 1952-01-14']
data=[]
for p in m['Parties']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['party']
    d['Típus']='Párt'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
    d=dict(d)
    d['Csoportok']=p['judet']
    d['Típus']='Megye'
    data.append(d)
for p in m['Countries']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['country']
    d['Típus']='Ország'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
for p in m['Groups']:
    start=pd.to_datetime(p['start'])
    end=pd.to_datetime(p['end'])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=p['group'].replace('Comisia pentru','').replace('Comisia','').strip().capitalize()
    d['Típus']='Bizottság'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
for p in m['Link']:
    start=pd.to_datetime(legs[p[-10:-6]][0])
    end=pd.to_datetime(legs[p[-10:-6]][1])    
    d={}
    d['Name']=m['Name']['full']
    d['Csoportok']=szen2[int(p[-1])-1]
    d['Típus']='Kamara'
    d['start']=start
    d['end']=end
    d['duration']=pd.to_timedelta(end-start).days
    data.append(d)
ef=pd.DataFrame(data)
eord=ef.groupby('Típus').sum()
eord['duration']=[1,60,30,4,50]
eord.columns=['order']
ef=ef.set_index('Típus').join(eord).reset_index()

In [167]:
ef

Típus                                          Csoportok  \
0   Bizottság                                   Politică externă   
1   Bizottság                                          Economică   
2   Bizottság                Învăţământ, ştiinţă, tineret, sport   
3   Bizottság         Politică economică, reformă şi privatizare   
4   Bizottság  Drepturile omului, culte şi problemele minorit...   
5   Bizottság                                   Politică externă   
6   Bizottság                            Buget, finanţe şi bănci   
7      Kamara                                 Camera Deputaților   
8      Kamara                                 Camera Deputaților   
9      Kamara                                 Camera Deputaților   
10     Kamara                                 Camera Deputaților   
11     Kamara                                              Senat   
12     Kamara                                 Camera Deputaților   
13     Kamara                                              Senat   
14      Megye                                              ILFOV   
15      Megye                                               ARAD   
16      Megye                                          BUCUREŞTI   
17      Megye                                          BUCUREŞTI   
18      Megye                                          BUCUREŞTI   
19      Megye                                          BUCUREŞTI   
20      Megye                                          BUCUREŞTI   
21      Megye                                          BUCUREŞTI   
22     Ország                                    Republica Elenă   
23     Ország                                 Republica Filipine   
24     Ország                         Republica Franceză - Senat   
25     Ország                                 Republica Filipine   
26     Ország                                    Republica Elenă   
27       Párt                                                PNL   
28       Párt                                                PNL   
29       Párt                                                PNL   
30       Párt                                                PNL   
31       Párt                                                PNL   
32       Párt                                        Independent   
33       Párt                                                PNL   
34       Párt                                               ALDE   

                                        Name  duration        end      start  \
0   Călin Constantin Anton POPESCU-TĂRICEANU      1462 2012-12-20 2008-12-19   
1   Călin Constantin Anton POPESCU-TĂRICEANU       850 1992-10-16 1990-06-19   
2   Călin Constantin Anton POPESCU-TĂRICEANU       850 1992-10-16 1990-06-19   
3   Călin Constantin Anton POPESCU-TĂRICEANU      1464 2000-11-30 1996-11-27   
4   Călin Constantin Anton POPESCU-TĂRICEANU      1455 2008-12-13 2004-12-19   
5   Călin Constantin Anton POPESCU-TĂRICEANU       436 2014-03-01 2012-12-20   
6   Călin Constantin Anton POPESCU-TĂRICEANU      1446 2004-11-30 2000-12-15   
7   Călin Constantin Anton POPESCU-TĂRICEANU      1462 2012-12-20 2008-12-19   
8   Călin Constantin Anton POPESCU-TĂRICEANU       850 1992-10-16 1990-06-19   
9   Călin Constantin Anton POPESCU-TĂRICEANU      1464 2000-11-30 1996-11-27   
10  Călin Constantin Anton POPESCU-TĂRICEANU      1455 2008-12-13 2004-12-19   
11  Călin Constantin Anton POPESCU-TĂRICEANU      1462 2016-12-21 2012-12-20   
12  Călin Constantin Anton POPESCU-TĂRICEANU      1446 2004-11-30 2000-12-15   
13  Călin Constantin Anton POPESCU-TĂRICEANU       922 2019-07-01 2016-12-21   
14  Călin Constantin Anton POPESCU-TĂRICEANU      1462 2012-12-20 2008-12-19   
15  Călin Constantin Anton POPESCU-TĂRICEANU       850 1992-10-16 1990-06-19   
16  Călin Constantin Anton POPESCU-TĂRICEANU      1464 2000-11-30 1996-11-27   
17  Călin Constantin Anton POPESCU-TĂRICEANU      1455 2008-12-13 2004-12-19   
18  Călin Constantin Anton POPESCU-TĂRI

In [168]:
alt.Chart(ef,
          title=m['Name']['full']
         ).mark_bar().encode(
    x=alt.X('start:T',title=''),
    x2='end:T',
    y=alt.Y('Csoportok:N',title='',
            sort=alt.EncodingSortField(field="order", op="mean", order='descending')),
    color=alt.Color('Típus:N',
        sort=alt.EncodingSortField(field="order", op="mean", order='descending'))
)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Save JSON

In [169]:
def party_normalizer(party):
    if party in ['N/A','NA','NI',['NA', 'NI'],'-','Independent','Minorități',
                'Fără adeziune la formaţiunea politică pentru care a candidat la alegeri']: return '👤'
    elif party in ['ALDE RO','ALDE Romania','Partidul Conservator',
                   'PC','Partidul Puterii Umaniste','ALDE','PUNR','PUR','PUR-SL']: return '🕊️'
    elif party in ['Forumul Democrat al Germanitor din România','FDGR']: return '⚫️'
    elif party in ['Partidul Naţional Liberal','PNL-CD',"PL'93",'PNL-AT','PAC',
        'Partidul Liberal Democrat','PNL']: return'📐'
    elif party in ['Partidul Democrat','Partidul Democrat-Liberal',
        'PD-L','PD','PDL']: return'🔶'
    elif party in ['Partidul Mișcarea Populară','PMP']: return '🍏'
    elif party in ['Partidul Naţional Ţaranesc Creştin Democrat','PNȚCD','PNTCD']:return '✳️'
    elif party in ['Partidul România Mare','PRM']:return '🔱'
    elif party in ['FSN','FDSN']:return '⚪️'
    elif party in ['USR']:return '🔷'
    elif party in ['PP-DD']:return '⚛️'
    elif party in ['PER','MER','PDAR','FER']:return '🍀'
    elif party in ['PSD','Partidul Social Democrat','Partidul Social Democrat + Partidul Conservator',
                   'PDSR','PSDR','PSM']:return '🌹'
    elif party in ['Romániai Magyar Demokrata Szövetség',
        'Uniunea Democrată Maghiară din România','RMDSZ','UDMR']:return '🌷'
    elif party in ['Uniunea Națională pentru Progresul României','UNPR']: return '🦅'
    else: 
        return '🔘'

In [170]:
def party_writer(party):
    if party in ['Romániai Magyar Demokrata Szövetség',
        'Uniunea Democrată Maghiară din România','RMDSZ','UDMR']:return 'RMDSZ'
    elif party in ['Fără adeziune la formaţiunea politică pentru care a candidat la alegeri']:
        return 'Independent'
    else: 
        return party

In [171]:
#megyek=json.loads(open('E:/Skydrive/Github/universal/ro/hun_megyek.json','r').read())
megyek=json.loads(open('C:/users/csala/Onedrive/Github/universal/ro/hun_megyek.json','r').read())

In [172]:
r={
"Buget, finanţe şi bănci":"Pénzügy",
"Drepturile omului, culte şi problemele minorităţilor naţionale":"Emberi jogok, kisebbségek",
"Economică":"Gazdaság, ipar",
"Politică economică, reformă şi privatizare":"Gazdaság, ipar",
"Politică externă":"Külpolitika, diaszpóra",
"Învăţământ, ştiinţă, tineret, sport":"Oktatás, tudomány, sport",
"Cultură, arte, mijloace de informare în masă":"Művészet, média",
"Juridică, de disciplină şi imunităţi":"Állami ügyek",
"Regulament":"Állami ügyek",
"Agricultură, silvicultură, industrie alimentară şi servicii specifice":"Mezőgazdaság, erdők",
"Buget şi finanţe":"Pénzügy",
"Industrii şi servicii":"Gazdaság, ipar",
"Tehnologia informaţiei şi comunicaţiilor":"Infrastruktúra, energia, kommunikáció",
"Afaceri europene":"Európa",
"Agricultură, silvicultură şi dezvoltare rurală":"Mezőgazdaság, erdők",
"Muncă, familie şi protecţie socială":"Munka, szociális és családi ügyek",
"Cultură, artă şi mijloace de informare în masă":"Művészet, média",
"Drepturile omului, culte şi minorităţi":"Emberi jogok, kisebbségek",
"Muncă şi protecţie socială":"Munka, szociális és családi ügyek",
"Buget, finanţe, activitate bancară şi piaţă de capital":"Pénzügy",
"Economică, industrii şi servicii":"Gazdaság, ipar",
"Privatizare şi administrarea activelor statului":"Állami ügyek",
"Învăţământ, ştiinţă, tineret şi sport":"Oktatás, tudomány, sport",
"Administraţie publică, organizarea teritoriului şi protecţia mediului":"Közigazgatás, környezetvédelem",
"Egalitatea de şanse":"Munka, szociális és családi ügyek",
"Specială pentru modificarea şi completarea regulamentului senatului":"Állami ügyek",
"Comunităţile de români din afara graniţelor ţării":"Külpolitika, diaszpóra",
"Cercetarea abuzurilor, corupţiei şi pentru petiţii":"Állami ügyek",
"Dezvoltare și strategie economică":"Gazdaság, ipar",
"Transporturi şi energie":"Infrastruktúra, energia, kommunikáció",
"Transporturi şi infrastructură":"Infrastruktúra, energia, kommunikáció",
"Sănătate publică":"Egészésgügy",
"Sănătate şi familie":"Egészésgügy",
"Apărare, ordine publică şi siguranţă naţională":"Biztonság",
"Egalitatea de şanse pentru femei şi bărbaţi":"Munka, szociális és családi ügyek",
"Echilibru ecologic şi protecţia mediului":"Közigazgatás, környezetvédelem",
"Industrie şi servicii":"Gazdaság, ipar",
"Mediu şi echilibru ecologic":"Közigazgatás, környezetvédelem",
"Cercetarea abuzurilor, combaterea corupţiei şi petiţii":"Állami ügyek",
"Muncă, sănătate, protecţie socială şi statutul femeii în societate":"Munka, szociális és családi ügyek",
"Administraţie centrală şi locală, amenajarea teritoriului şi urbanism":"Közigazgatás, környezetvédelem",
"Administraţie publică şi amenajarea teritoriului":"Közigazgatás, környezetvédelem",
"Agricultură, silvicultură, industrie alimentară şi servicii pentru agricultură":"Mezőgazdaság, erdők",
"Juridică, de numiri, disciplină, imunităţi şi validări":"Állami ügyek",
"Subcomisia pentru spaţiu":"Közigazgatás, környezetvédelem",
"Cercetarea abuzurilor şi pentru petiţii":"Állami ügyek",
"Muncă, probleme sociale, sănătate":"Munka, szociális és családi ügyek",
"Mediu":"Közigazgatás, környezetvédelem",
"Economică şi pentru problemele de industrie, comerţ, transporturi, telecomunicaţii, servicii":"Infrastruktúra, energia, kommunikáció",
"Constituţionalitate, libertăţi civile şi monitorizare a executării hotărârilor curţii europene a drepturilor omului":"Európa",
"De politică şi pentru drepturile fundamentale ale omului şi cetăţeanului":"Emberi jogok, kisebbségek",
"Românii de pretutindeni":"Külpolitika, diaszpóra",
"Subcomisia pentru monitorizarea executării hotărârilor curţii europene a drepurilor omului":"Emberi jogok, kisebbségek",
"Subcomisia parlamentară pentru romi":"Emberi jogok, kisebbségek",
"Agricultură, industrie alimentară, silvicultură şi protecţia mediului înconjurător":"Mezőgazdaság, erdők",
"Subcomisia pentru protecţia copilului":"Munka, szociális és családi ügyek",
"Subcomisia pentru minoritatea maghiară":"Emberi jogok, kisebbségek",
"Artă, cultură şi mijloace de informare în masă":"Művészet, média",
"Subcomisia pentru demnitate umană":"Emberi jogok, kisebbségek",
"Subcomisia pentru egalitate de tratament şi  nediscriminare":"Emberi jogok, kisebbségek",
"Subcomisia pentru problemele românilor de pretutindeni":"Külpolitika, diaszpóra",
"Subcomisia pe problemele românilor din afara graniţelor":"Külpolitika, diaszpóra",
"Politică financiară, bancară şi bugetară":"Pénzügy",
"Permanentă comună a camerei deputaţilor şi senatului pentru relaţia cu unesco":"Külpolitika, diaszpóra",
"Afaceri europene a parlamentului româniei":"Európa",
"Parlamentară a revoluţionarilor din decembrie 1989":"Állami ügyek",
"Parlamentului româniei pentru integrare europeană":"Európa",
"De redactare a proiectului constituţiei româniei":"Állami ügyek",
"Permanentă a camerei deputaţilor şi senatului privind statutul deputaţilor şi al senatorilor, organizarea şi funcţionarea şedinţelor comune ale camerei deputaţilor şi senatului":"Állami ügyek",
"Comună pentru integrare europeană dintre parlamentul româniei și parlamentul republicii moldova":"Európa",
"Comună permanentă a camerei deputaţilor şi senatului pentru exercitarea controlului parlamentar asupra activităţii sri":"Biztonság",
"Apărare şi asigurarea ordinii publice":"Biztonság",
"Parlamentară specială pentru controlul activităţii serviciului de informaţii externe":"Biztonság"}

In [173]:
r2={"Biztonság":"🛃 Biztonság",
"Egészésgügy":"♿️ Egészségügy",
"Emberi jogok, kisebbségek":"🚹 Emberi jogok, kisebbségek",
"Európa":"⏺️ Európa",
"Gazdaság, ipar":"⏫ Gazdaság, ipar",
"Infrastruktúra, energia, kommunikáció":"➿ Infrastruktúra, energia, kommunikáció",
"Közigazgatás, környezetvédelem":"🔄 Közigazgatás, környezetvédelem",
"Külpolitika, diaszpóra":"🌐 Külpolitika, diaszpóra",
"Mezőgazdaság, erdők":"🚰 Mezőgazdaság, erdők",
"Munka, szociális és családi ügyek":"🚻 Munka, szociális és családi ügyek",
"Művészet, média":"🎦 Művészet, média",
"Oktatás, tudomány, sport":"🔠 Oktatás, tudomány, sport",
"Pénzügy":"💹 Pénzügy",
"Állami ügyek":"🔀 Állami ügyek"}

In [174]:
def extender(r,n=5):
    p={}
    for i in r:
        for j in range(n):
            p[i+j*' ']=r[i]+j*' '
    return p

In [175]:
cc={"":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/africadesud.gif":"za",
"/camera_deputatilor/grupuri_prietenie/flags1/albania.gif":"al",
"/camera_deputatilor/grupuri_prietenie/flags1/algeria.gif":"dz",
"/camera_deputatilor/grupuri_prietenie/flags1/arabiasaudi.gif":"sa",
"/camera_deputatilor/grupuri_prietenie/flags1/argentina.gif":"ar",
"/camera_deputatilor/grupuri_prietenie/flags1/armenia.gif":"am",
"/camera_deputatilor/grupuri_prietenie/flags1/australia.gif":"au",
"/camera_deputatilor/grupuri_prietenie/flags1/austria.gif":"at",
"/camera_deputatilor/grupuri_prietenie/flags1/azerbaidjan.gif":"az",
"/camera_deputatilor/grupuri_prietenie/flags1/bahrein.gif":"bh",
"/camera_deputatilor/grupuri_prietenie/flags1/belarus.gif":"by",
"/camera_deputatilor/grupuri_prietenie/flags1/belgia.gif":"be",
"/camera_deputatilor/grupuri_prietenie/flags1/bolivia.gif":"bo",
"/camera_deputatilor/grupuri_prietenie/flags1/bosnia.gif":"ba",
"/camera_deputatilor/grupuri_prietenie/flags1/brazilia.gif":"br",
"/camera_deputatilor/grupuri_prietenie/flags1/bulgaria.gif":"bg",
"/camera_deputatilor/grupuri_prietenie/flags1/canada.gif":"ca",
"/camera_deputatilor/grupuri_prietenie/flags1/cehia.gif":"cz",
"/camera_deputatilor/grupuri_prietenie/flags1/chile.gif":"cl",
"/camera_deputatilor/grupuri_prietenie/flags1/china.gif":"cn",
"/camera_deputatilor/grupuri_prietenie/flags1/cipru.gif":"cy",
"/camera_deputatilor/grupuri_prietenie/flags1/columbia.gif":"co",
"/camera_deputatilor/grupuri_prietenie/flags1/congo.gif":"cg",
"/camera_deputatilor/grupuri_prietenie/flags1/coreea.gif":"kr",
"/camera_deputatilor/grupuri_prietenie/flags1/costarica.gif":"cr",
"/camera_deputatilor/grupuri_prietenie/flags1/cotedivoir.gif":"ci",
"/camera_deputatilor/grupuri_prietenie/flags1/croatia.gif":"hr",
"/camera_deputatilor/grupuri_prietenie/flags1/cuba.gif":"cu",
"/camera_deputatilor/grupuri_prietenie/flags1/danemarca.gif":"dk",
"/camera_deputatilor/grupuri_prietenie/flags1/ecuador.gif":"ec",
"/camera_deputatilor/grupuri_prietenie/flags1/egipt.gif":"eg",
"/camera_deputatilor/grupuri_prietenie/flags1/elvetia.gif":"ch",
"/camera_deputatilor/grupuri_prietenie/flags1/emirate.png":"ae",
"/camera_deputatilor/grupuri_prietenie/flags1/estonia.gif":"ee",
"/camera_deputatilor/grupuri_prietenie/flags1/filipine.gif":"ph",
"/camera_deputatilor/grupuri_prietenie/flags1/finlanda.gif":"fi",
"/camera_deputatilor/grupuri_prietenie/flags1/franta.gif":"fr",
"/camera_deputatilor/grupuri_prietenie/flags1/georgia.gif":"ge",
"/camera_deputatilor/grupuri_prietenie/flags1/germania.gif":"de",
"/camera_deputatilor/grupuri_prietenie/flags1/grecia.gif":"gr",
"/camera_deputatilor/grupuri_prietenie/flags1/guineea.gif":"gn",
"/camera_deputatilor/grupuri_prietenie/flags1/india.gif":"in",
"/camera_deputatilor/grupuri_prietenie/flags1/indonezia.gif":"id",
"/camera_deputatilor/grupuri_prietenie/flags1/iordania.gif":"jo",
"/camera_deputatilor/grupuri_prietenie/flags1/irak.gif":"iq",
"/camera_deputatilor/grupuri_prietenie/flags1/iran.gif":"ir",
"/camera_deputatilor/grupuri_prietenie/flags1/irlanda.gif":"ie",
"/camera_deputatilor/grupuri_prietenie/flags1/islanda.gif":"is",
"/camera_deputatilor/grupuri_prietenie/flags1/israel.gif":"il",
"/camera_deputatilor/grupuri_prietenie/flags1/italia.gif":"it",
"/camera_deputatilor/grupuri_prietenie/flags1/iugoslavia.gif":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/japonia.gif":"jp",
"/camera_deputatilor/grupuri_prietenie/flags1/kazahtan.gif":"kz",
"/camera_deputatilor/grupuri_prietenie/flags1/kuwait.gif":"kw",
"/camera_deputatilor/grupuri_prietenie/flags1/letonia.gif":"lv",
"/camera_deputatilor/grupuri_prietenie/flags1/liban.gif":"lb",
"/camera_deputatilor/grupuri_prietenie/flags1/lituania.gif":"lt",
"/camera_deputatilor/grupuri_prietenie/flags1/luxemburg.gif":"lu",
"/camera_deputatilor/grupuri_prietenie/flags1/macedonia.gif":"mk",
"/camera_deputatilor/grupuri_prietenie/flags1/malaezia.gif":"my",
"/camera_deputatilor/grupuri_prietenie/flags1/malta.gif":"mt",
"/camera_deputatilor/grupuri_prietenie/flags1/mareabritanie.gif":"gb",
"/camera_deputatilor/grupuri_prietenie/flags1/maroc.gif":"ma",
"/camera_deputatilor/grupuri_prietenie/flags1/mexic.gif":"mx",
"/camera_deputatilor/grupuri_prietenie/flags1/mongolia.gif":"mn",
"/camera_deputatilor/grupuri_prietenie/flags1/muntenegru.gif":"me",
"/camera_deputatilor/grupuri_prietenie/flags1/norvegia.gif":"no",
"/camera_deputatilor/grupuri_prietenie/flags1/olanda.gif":"nl",
"/camera_deputatilor/grupuri_prietenie/flags1/oman.gif":"om",
"/camera_deputatilor/grupuri_prietenie/flags1/pakistan.gif":"pk",
"/camera_deputatilor/grupuri_prietenie/flags1/panama.gif":"pa",
"/camera_deputatilor/grupuri_prietenie/flags1/peru.gif":"pe",
"/camera_deputatilor/grupuri_prietenie/flags1/polonia.gif":"pl",
"/camera_deputatilor/grupuri_prietenie/flags1/portugalia.gif":"pt",
"/camera_deputatilor/grupuri_prietenie/flags1/qatar.gif":"qa",
"/camera_deputatilor/grupuri_prietenie/flags1/rusia.gif":"ru",
"/camera_deputatilor/grupuri_prietenie/flags1/senegal.gif":"sn",
"/camera_deputatilor/grupuri_prietenie/flags1/singapore.gif":"sg",
"/camera_deputatilor/grupuri_prietenie/flags1/siria.gif":"sy",
"/camera_deputatilor/grupuri_prietenie/flags1/slovacia.gif":"sk",
"/camera_deputatilor/grupuri_prietenie/flags1/slovenia.gif":"si",
"/camera_deputatilor/grupuri_prietenie/flags1/spania.gif":"es",
"/camera_deputatilor/grupuri_prietenie/flags1/srilanka.gif":"lk",
"/camera_deputatilor/grupuri_prietenie/flags1/sudan.gif":"sd",
"/camera_deputatilor/grupuri_prietenie/flags1/suedia.gif":"se",
"/camera_deputatilor/grupuri_prietenie/flags1/thailanda.gif":"th",
"/camera_deputatilor/grupuri_prietenie/flags1/tunisia.gif":"tn",
"/camera_deputatilor/grupuri_prietenie/flags1/turcia.gif":"tr",
"/camera_deputatilor/grupuri_prietenie/flags1/turkmenistan.gif":"tm",
"/camera_deputatilor/grupuri_prietenie/flags1/ucraina.gif":"ua",
"/camera_deputatilor/grupuri_prietenie/flags1/ungaria.gif":"hu",
"/camera_deputatilor/grupuri_prietenie/flags1/urss.gif":"uu",
"/camera_deputatilor/grupuri_prietenie/flags1/usa.png":"us",
"/camera_deputatilor/grupuri_prietenie/flags1/uruguay.gif":"uy",
"/camera_deputatilor/grupuri_prietenie/flags1/venezuela.gif":"ve",
"/camera_deputatilor/grupuri_prietenie/flags1/vietnam.gif":"vn",
"/camera_deputatilor/grupuri_prietenie/flags1/zambia.gif":"zm",
"/img/spacer.gif":"uu"}

In [176]:
#!pip install pycountry
import pycountry
import gettext
#!pip install emoji-country-flag
import flag
hun = gettext.translation('iso3166', pycountry.LOCALES_DIR,
                              languages=['hu'])
hun.install()

In [177]:
def local_country(c):
    if c=='uu':return 'Más ország'
    z=_(pycountry.countries.get(alpha_2=c.upper()).name)
    #manual overrides
    if z=='Koreai Köztársaság':z='Dél-Korea'
    if z=='Nagy-Britannia (Egyesült Királyság)':z='Egyesült Királyság'
    if z=='Macedón Köztársaság':z='Macedónia'
    if z=='Szíriai Arab Köztársaság':z='Szíria'
    if z=='Venezuelai Bolivári Köztársaság':z='Venezuela'
    if z=='Orosz Föderáció':z='Oroszország'
    g=flag.flagize(':'+c.upper()+':')
    return g+' '+z

In [178]:
def tolerance_cleaner(ef,dt=30):
    hfs=[]
    df=ef.sort_values('start').groupby(['Típus','Csoportok','start','end']).first()
    df=df.reset_index(level=[2,3], drop=False)
    for p in df.index.get_level_values(0).unique():
        for r in df.loc[p].index.get_level_values(0).unique():
            hf=df.loc[p].loc[[r]].sort_values('start')
            hf['dt']=list((hf[1:]['start']-hf['end'][:-1]).dt.days.values)+[0]
            hf=hf.reset_index(drop=False)
            new_column=[]
            counters={i:0 for i in hf['Csoportok'].unique()}
            for i in hf.index:
                j=hf.loc[i]['Csoportok']
                new_column.append(j+counters[j]*' ')
                if hf.loc[i]['dt']>dt:
                    counters[j]+=1
            hf['Csoportok1']=new_column
            hf['Típus']=p
            hf=hf.drop(['dt'],axis=1)
            hfs.append(hf)
    hfs=pd.concat(hfs)
    hfs=hfs.reset_index(drop=True)
    return(hfs)

In [179]:
efs=[]
for n in nmembers:
    m=nmembers[n]
    data=[]
    for p in m['Parties']:
        start=pd.to_datetime(p['start'])
        end=pd.to_datetime(p['end'])    
        d={}
        d['Name']=m['Name']['full']
        d['Csoportok']=party_normalizer(p['party'])+' '+party_writer(p['party'])+' - '+img['party'][party_writer(p['party'])]['name']
        d['Típus']='Párt'
        d['Image']=img['party'][p['party']]['img']
        d['start']=start
        d['end']=end
        d['duration']=pd.to_timedelta(end-start).days
        data.append(d)
        d=dict(d)
        d['Csoportok']='🇷🇴 '+megyek[p['judet'].capitalize()]
        d['Típus']='Megye'
        data.append(d)
    for p in m['Countries']:
        start=pd.to_datetime(p['start'])
        end=pd.to_datetime(p['end'])    
        d={}
        d['Name']=m['Name']['full']
        d['Csoportok']=p['country']
        d['Típus']='Ország'
        d['Image']=img['state'][p['country']]
        d['start']=start
        d['end']=end
        d['duration']=pd.to_timedelta(end-start).days
        data.append(d)
    for p in m['Groups']:
        start=pd.to_datetime(p['start'])
        end=pd.to_datetime(p['end'])    
        d={}
        d['Name']=m['Name']['full']
        d['Csoportok']=p['group'].replace('Comisia pentru','').replace('Comisia','').strip().capitalize()
        d['Típus']='Bizottság'
        d['start']=start
        d['end']=end
        d['duration']=pd.to_timedelta(end-start).days
        data.append(d)
    for pi in range(len(m['Link'])):
        p=m['Link'][pi]
        start=pd.to_datetime(legs[p[-10:-6]][0])
        end=pd.to_datetime(legs[p[-10:-6]][1])    
        d={}
        d['Name']=m['Name']['full']
        d['Csoportok']=szen[int(p[-1])-1] #hu
        #d['Csoportok']=szen2[int(p[-1])-1] #ro
        d['Típus']='Kamara'
        d['start']=start
        d['end']=end
        d['duration']=pd.to_timedelta(end-start).days
        data.append(d)
    ef=pd.DataFrame(data)
    ef=ef.set_index('Típus').reset_index()
    ef['Simple']=n
    
    #committees
    ef=ef.replace(r).replace(r2)
    
    #countries
    if 'Image' in ef.columns:
        cz={}
        cs=ef[ef['Típus']=='Ország'].groupby('Image').first()['Csoportok']
        for i in sorted(ef[ef['Típus']=='Ország']['Image'].unique()):
            cz[cs.loc[i]]=local_country(cc[i])

        #manual overrides
        cz['Republica Macedonia']='🇲🇰 Macedónia'
        cz['Republica Federală Iugoslavia']='Jugoszlávia'
        cz['fosta Republică Iugoslavă Macedonia']='🇲🇰 Macedónia'
        cz['Republica Franceză - Senat']='🇫🇷 Franciaország'
        cz['Republica Serbia']='🇷🇸 Szerbia'
        cz['Republica Franceză - Adunarea Naţională']='🇫🇷 Franciaország'
        ef=ef.replace(cz)
    
    #unique counting
    new_column=[]
    counters={i:0 for i in ef['Csoportok'].unique()}
    for i in ef['Csoportok']:
        new_column.append(i+counters[i]*' ')
        counters[i]+=1
    ef['Csoportok0']=new_column
    
    #tolerance
    ef=tolerance_cleaner(ef).reset_index()
    
    #append
    efs.append(ef)

In [180]:
efs2=pd.concat(efs)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [185]:
efs4=efs2[['start', 'end', 'Name', 'duration',
       'Simple', 'Csoportok','Csoportok0','Csoportok1', 'Típus']]

In [186]:
efs4['Name']=efs4['Name'].str.replace('chestor al Camerei Deputaţilor','')\
    .str.replace('vicepreşedinte al Camerei Deputaţilor','')\
    .str.replace('secretar al Camerei Deputaţilor','')\
    .str.replace('Preşedintele Senatului','')\
    .str.replace('Preşedintele Camerei Deputaţilor','')\
    .str.replace('secretar al Senatului','')\
    .str.replace('chestor al Senatului','')\
    .str.replace('vicepreşedinte al Senatului','')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [187]:
j=efs4.reset_index().drop('index',axis=1).T.astype(str).to_dict()
open('ignore/member_timelines.json','w').write(json.dumps(list(j.values())))

10875912

In [188]:
import zipfile

with zipfile.ZipFile("json/member_timelines.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_timelines.json","member_timelines.json")

In [189]:
members={}
simple={}
simple2={}
for i in efs4[['Name','Simple']].T.iteritems():
    if i[1][0] not in simple: simple[i[1][0]]=i[1][1]
    if i[1][1] not in simple2: simple2[i[1][1]]=i[1][0]
members['s']=simple
members['s2']=simple2

In [190]:
members['megye']=efs4[efs4['Típus']=='Megye'].sort_values(by='start')[['Simple','Csoportok']].set_index('Csoportok')\
    .drop_duplicates(keep='last').reset_index().set_index('Simple')['Csoportok'].to_dict()

In [191]:
members['part']=efs4[efs4['Típus']=='Párt'].sort_values(by='start')[['Simple','Csoportok']].set_index('Csoportok')\
    .drop_duplicates(keep='last').reset_index().set_index('Simple')['Csoportok'].to_dict()

In [192]:
members['deaths']=deaths

Vega direct plot

In [193]:
m='Popescu-Tariceanu Calin Constantin Anton | 1952-01-14'
ef=efs4[efs4['Simple']==m]
viz=alt.Chart(ef,
          title=m
         ).mark_bar().encode(
    x=alt.X('start:T',title=''),
    x2='end:T',
    y=alt.Y('Csoportok:N',title='',
            sort=alt.EncodingSortField(field="order", op="mean", order='descending')),
    color=alt.Color('Típus:N',
        sort=alt.EncodingSortField(field="order", op="mean", order='descending'))
)
# viz.save('json/viz.json')
viz.interactive()

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Member images

In [194]:
photos={name:nmembers[name]['Photo'][np.array([p[-10:-6] for p in nmembers[name]['Link']]).argmax()] for name in nmembers}

Download photos if you need to..

In [ ]:
import urllib.request

In [ ]:
#downloader - run once only
for p in photos:
    urllib.request.urlretrieve(photos[p], "photos3/"+p.replace('|','+')+'.jpg')

In [ ]:
from PIL import Image
#!pip install python-resize-image
from resizeimage import resizeimage

In [ ]:
err=[]
for p in photos:
    try:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img = resizeimage.resize_cover(img, [123,150])
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)
    except:
        print(p)
        err.append(p)

In [ ]:
#run this then iterate cell above until no errors
for p in err:
    urllib.request.urlretrieve(photos[p], "photos3/"+p.replace('|','+')+'.jpg')

In [ ]:
for p in err:
    try:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img = resizeimage.resize_cover(img, [123,150])
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)
    except:
        img = Image.open('photos3/'+p.replace('|','+')+'.jpg', 'r')
        img.save('photos4/'+p.replace('|','+')+'.jpg', img.format)

Otherwise continue here:

In [195]:
members['photo']={p:'photos4/'+p.replace('|','+')+'.jpg' for p in photos}

In [196]:
open('ignore/member_simple.json','w').write(json.dumps(members))

974747

In [197]:
import zipfile

with zipfile.ZipFile("json/member_simple.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_simple.json","member_simple.json")

In [198]:
pretty={'- din care declaraţii politice':'🧾 Nyilatkozatok',
        'Declaraţii politice depuse în scris':'🧾 Nyilatkozatok',
       'Luari de cuvânt':'🎙️ Felszólalások', 
        'Luari de cuvânt în BP':'🎙️ Felszólalások', 
        'Motiuni':'📚 Javaslatok', 
        'Proiecte de hotarâre initiate':'📚 Javaslatok', 
        'Propuneri legislative initiate':'📚 Javaslatok',
        'Sedințe':'💺 Ülések', 
        'Întrebari si interpelari':'🎙️ Felszólalások'}

In [199]:
efx=[]
for n in nmembers:
    m=nmembers[n]
    data=[]
    for pi in range(len(m['Link'])):
        p=m['Link'][pi]
        d={}
        d['Simple']=n
        d['Link']=p
        d['Session']=p[-10:-6]
        for a in m['Activity']:
            if type(m['Activity'][a])==list:
                g=m['Activity'][a]
            else:
                g=[m['Activity'][a]]
            for h in g:
                if h['start']==legs[d['Session']][0]: 
                    if pretty[a] not in d:d[pretty[a]]=0
                    d[pretty[a]]+=h['value']
        data.append(d)
    ef=pd.DataFrame(data)
    
    #append
    efx.append(ef)

In [200]:
efx2=pd.concat(efx).fillna(0).reset_index().drop('index',axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [201]:
j=efx2.T.astype(str).to_dict()
j2=list(j.values())

In [202]:
j3={}
for i in j2:
    if i['Simple'] not in j3:j3[i['Simple']]=[]
    j3[i['Simple']].append(i)

In [203]:
open('ignore/member_activity.json','w').write(json.dumps(j3))

1433479

In [204]:
import zipfile

with zipfile.ZipFile("json/member_activity.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/member_activity.json","member_activity.json")